# Modeling

In [93]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

## Load In Data

### Y Data

In [58]:
#X Data
with open('X.pkl','rb') as f:
    X = pickle.load(f)
    print(len(X))
    
#Y Data
with open('Y.pkl','rb') as f:
    Y = pickle.load(f)
    print(len(Y))


51
51


In [144]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [145]:
X_train[0]

array([[-8.9657059e+00, -8.9464569e+00, -9.0931015e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-4.2739410e+00, -4.2424545e+00, -4.2462006e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 2.7397156e-02,  3.3378601e-02, -5.4862976e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [-1.2730175e-02, -5.8371192e-01,  5.4546839e-01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.3081874e-01, -7.4142410e-04, -5.0024819e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-3.5824046e-02,  8.4550835e-02, -8.5358843e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [146]:
def Scaler(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = []
    X_test_scaled = []
    for i in range(38):
        scaler.fit(X_train[i])
        X_train_scaled.append(scaler.transform(X_train[i]))
    for i in range(13):
        X_test_scaled.append(scaler.transform(X_test[i]))
    return np.array(X_train_scaled), np.array(X_test_scaled)

In [147]:
y_test[0].todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]])

In [148]:
X_train_scaled, X_test_scaled = Scaler(X_train, X_test)
print("X Train Shape: ", X_train_scaled.shape)
print("X Test Shape: ", X_test_scaled.shape)
print("Number of features: ", len(X[0]))
print("Number of observations: ", len(X[0][0]))

X Train Shape:  (38, 12, 32409)
X Test Shape:  (13, 12, 32409)
Number of features:  12
Number of observations:  32409


In [149]:
test_shape = tf.reshape(X_train_scaled, [38, 12, 32409])

In [150]:
y_train[0].todense().shape

(32409, 19)

# Construct a Baseline Model

In [125]:
model = Sequential()
model.add(LSTM(2, activation='relu', input_shape=(32409, 12)))
model.add(Dense(19, activation="softmax"))
model.compile(optimizer='adam', loss='mse')

In [123]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_22 (LSTM)               (None, 2)                 176       
_________________________________________________________________
dense_12 (Dense)             (None, 19)                57        
Total params: 233
Trainable params: 233
Non-trainable params: 0
_________________________________________________________________


In [152]:
# This is a list comprehesnion to switch the order of X_train, not sure if it is necessary or not
#X_train = [x.reshape(32409, 12) for x in X_train]

In [132]:
X_train[0].reshape(32409,12).shape

(32409, 12)

# Praveen
- I'm trying to construct a simple LSTM model as my baseline
- most common error is a dimesionality error (ex: "expected ndim=3, found ndim=2"


In [139]:
model.fit(X_train[0], y_train[0], epochs=10, batch_size=5,verbose=1)

Epoch 1/10


ValueError: in user code:

    /Users/weatherford/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/weatherford/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/weatherford/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/weatherford/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/weatherford/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _

In [ ]:
tf.keras.layers.InputLayer(
    input_shape=None,
    batch_size=None,
    dtype=None,
    input_tensor=None,
    sparse=None,
    name=None,
    ragged=None,
    type_spec=None,
    **kwargs
)